In [2]:
import csv
import numpy as np
import os
import pandas as pd
import time

In [2]:
gold_drug = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/7_2_gold_drug_feature_id.tsv")

In [3]:
gold_drug_id = gold_drug['DrugBank_ID'].drop_duplicates()
print len(gold_drug_id)

471


In [4]:
drugbank = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/drug_set.txt", header=None)
drugbank = drugbank.rename(columns = {0:'DrugBank_ID', 1:'Drug_Name', 2:'InChI', 3:'SMILES', 4:'State'})
drugbank.head(1)

,DrugBank_ID,Drug_Name,InChI,SMILES,State
0,DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,approved


In [5]:
# gold drug의 순서대로 drugbank를 정렬하기.
df_gold_drug_smiles = drugbank[drugbank['DrugBank_ID'].isin(gold_drug_id)]
sorter = gold_drug_id.tolist()
df_gold_drug_smiles.DrugBank_ID = df_gold_drug_smiles.DrugBank_ID.astype("category")
df_gold_drug_smiles.DrugBank_ID.cat.set_categories(sorter, inplace=True)
df_gold_drug_smiles = df_gold_drug_smiles.sort_values(["DrugBank_ID"])
print len(df_gold_drug_smiles)
df_gold_drug_smiles.head(2) # drug bank id로 중복체크 했을 때 중복 없었음.

456


/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,DrugBank_ID,Drug_Name,InChI,SMILES,State
103,DB00215,Citalopram,InChI=1S/C20H21FN2O/c1-23(2)11-3-10-20(17-5-7-...,CN(C)CCCC1(OCC2=C1C=CC(=C2)C#N)C1=CC=C(F)C=C1,approved
965,DB01327,Cefazolin,InChI=1S/C14H14N8O4S3/c1-6-17-18-14(29-6)28-4-...,[H][C@]12SCC(CSC3=NN=C(C)S3)=C(N1C(=O)[C@H]2NC...,approved


In [9]:
import pybel
df_sim = pd.DataFrame()
drug_id = df_gold_drug_smiles['DrugBank_ID'].tolist()
for i in drug_id:
    smi1 = df_gold_drug_smiles[df_gold_drug_smiles.DrugBank_ID == i]['SMILES'].tolist()[0]
    for j in drug_id:
        smi2 = df_gold_drug_smiles[df_gold_drug_smiles.DrugBank_ID == j]['SMILES'].tolist()[0]
        
        smiles = [smi1, smi2]
        mol = [pybel.readstring('smi',x) for x in smiles]
        fps = [x.calcfp(fptype='FP2') for x in mol]
        sim = fps[0] | fps[1]
        
        temp_df = pd.DataFrame([[i, j, sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'Similarity_Score'])
        df_sim = df_sim.append(temp_df)

In [36]:
df_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv", sep='\t', index=False)
df_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv", sep='\t', index=False)

In [40]:
df_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv")

In [8]:
gold_drug_matrix = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/8_gold_standard_drug_binary_matrix.tsv", index_col=0)

In [9]:
gold_drug_matrix.head()

,DB00215,DB01327,DB00537,DB00175,DB00181,DB00257,DB00343,DB01128,DB00678,DB00808,...,DB01098,DB01033,DB01254,DB01261,DB00323,DB00287,DB00905,DB01088,DB01268,DB01590
C0000737,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,0,0,0,1
C0000786,1,0,0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
C0001824,1,1,1,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
C0002170,1,0,1,1,1,1,1,1,1,0,...,1,0,0,1,1,0,0,0,0,0
C0002453,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics import jaccard_similarity_score
df_jac = pd.DataFrame()
drug_id = df_gold_drug_smiles['DrugBank_ID'].tolist()
for i in drug_id:
    desc1 = gold_drug_matrix[i]
    for j in drug_id:
        desc2 = gold_drug_matrix[j]
        
        jac_sim = jaccard_similarity_score(desc1,desc2)
        
        temp_df = pd.DataFrame([[i, j, jac_sim]], columns = ['DrugBank_ID1','DrugBank_ID2', 'Similarity_Score'])
        df_jac = df_jac.append(temp_df)

In [31]:
df_jac.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_phenotype.tsv", sep='\t', index=False)
df_jac.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_phenotype.tsv", sep='\t', index=False)

In [32]:
df_jac = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_phenotype.tsv")

In [3]:
############ sequence based similarity
drug_target = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/human2.txt", header=None)
target_sequence = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/protein_seq.txt")

In [4]:
drug_target = drug_target.rename(columns={0:'DrugBank_ID', 1:'Target'})
print len(drug_target)
drug_target.head(2)

10764


,DrugBank_ID,Target
0,DB00114,P19113
1,DB00117,P19113


In [5]:
print len(target_sequence)
target_sequence.head(2)

2290


,Entry,Sequence,Length
0,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...,662.0
1,Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...,602.0


In [6]:
drug_seq = pd.merge(left=drug_target, right=target_sequence, left_on='Target', right_on='Entry')
print len(drug_seq)
drug_seq.head(2)

10764


,DrugBank_ID,Target,Entry,Sequence,Length
0,DB00114,P19113,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...,662.0
1,DB00117,P19113,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...,662.0


In [ ]:
# gold drug 

In [54]:
import swalign
# choose your own values here… 2 and -1 are common.
match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)

sw = swalign.LocalAlignment(scoring)  # you can also choose gap penalties, etc...
alignment = sw.align('AAAAAA','AAAAAA')
alignment.dump()

Query: 1 AAAAAA 6
         ||||||
Ref  : 1 AAAAAA 6

Score: 12
Matches: 6 (100.0%)
Mismatches: 0
CIGAR: 6M


In [165]:
import swalign
import math
from itertools import product

match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)
sw = swalign.LocalAlignment(scoring)  # you can also choose gap penalties, etc...

df_sw = pd.DataFrame()
drug_id = df_gold_drug_smiles['DrugBank_ID'].tolist()
gold_drug_seq = drug_seq[drug_seq['DrugBank_ID'].isin(drug_id)]
print time.strftime('%a %H:%M:%S')
for i in drug_id:
    seqs1 = gold_drug_seq[gold_drug_seq.DrugBank_ID == i]['Sequence'].tolist()
    for j in drug_id:
        if i==j:
            continue
        seqs2 = gold_drug_seq[gold_drug_seq.DrugBank_ID == j]['Sequence'].tolist()
        query = list(product(seqs1, seqs2))
        
        scores = list()
        for k in query:
            seq1 = k[0]
            seq2 = k[1]
            sw_score = sw.align(seq1,seq2).score / (math.sqrt(sw.align(seq1, seq1).score)* math.sqrt(sw.align(seq2, seq2).score))
            scores.append(sw_score)
        final_score = max(scores) / len(scores)
        temp_df = pd.DataFrame([[i, j, final_score]], columns = ['DrugBank_ID1','DrugBank_ID2', 'Similarity_Score'])
        df_sw = df_sw.append(temp_df)
print time.strftime('%a %H:%M:%S')

Sun 15:44:25


KeyboardInterrupt: 

In [ ]:
중단한 시간은 수요일 오후 3시 40분인데 하다 만 갯수는 11개뿐.

In [166]:
i

'DB01136'

In [167]:
drug_id.index(i)

11

In [ ]:
#문제가 많음.
1. 중복문제. a-b , b-a 가 제대로 삭제 되지 않은채로 평균값을 구하고 있음.
2. complexity가 너무 높음.